In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
# Import the 'loadmat' function from scipy's io module.
# This function allows us to read MATLAB files in Python.
from scipy.io import loadmat

In [41]:
path = r"G:\AI学习资料\machine-learning-2014\机器学习课程2014源代码\python代码\ex3-neural network\ex3data1.mat"
raw_data = loadmat(path)
print(raw_data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011', '__version__': '1.0', '__globals__': [], 'X': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'y': array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)}


In [71]:
# Define a function 'sigmoid' to compute the sigmoid of 'z'.
def sigmoid(z):
    return 1/(1+np.exp(-z))

# Define a function 'get_x' to get the feature matrix 'X' from 'raw_data'.
# 'raw_data' is a dictionary containing the data.
def get_x(raw_data):
    # 'X' is the feature matrix.
    X = raw_data["X"]
    # Insert a column of ones at the beginning of 'X' for the intercept term.
    X = np.insert(X,obj=0,values=1,axis=1)
    return X

# Define a class 'Get_y' to get the target variable 'y' for a specific class.
class Get_y(object):
    # The constructor takes 'raw_data' as an argument and stores it in the 'data' attribute.
    def __init__(self, raw_data):
        self.data = raw_data
    # Define a method 'get_y' to get the target variable 'y' for the class 'num'.
    # 'num' is the class label.
    def get_y(self, num):
        # 'y' is the target variable.
        y = self.data["y"]
        # Replace 10 with 0 in 'y'.
        y[y==10]=0
        # Return 'y' as a binary variable where 1 indicates the class 'num' and 0 indicates the other classes.
        return np.where(y==num,1,0)

# 'X' is assigned the feature matrix obtained from 'raw_data'.
X = get_x(raw_data)
# 'Y' is an instance of the 'Get_y' class initialized with 'raw_data'.
Y = Get_y(raw_data)

In [67]:
print(f"X.shape: {X.shape},  Y.get_y(0).shape: {Y.get_y(0).shape}")

X.shape: (5000, 401),  Y.get_y(0).shape: (5000, 1)


In [69]:
# 'theta' is initialized as a zero vector with the same length as the number of features in 'X'.
theta = np.zeros(X.shape[1])
theta.shape

(401,)

In [86]:
# Define a function 'cost' to compute the regularized cost function for logistic regression.
# 'theta' is the parameter vector.
# 'X' is the feature matrix.
# 'Y' is an instance of the 'Get_y' class.
# 'y_num' is the class label.
# 'lamda' is the regularization parameter.
def cost(theta,X,Y,y_num,lamda=1):
    # 'm' is the number of training examples.
    m = X.shape[0]
    # 'y' is the target variable for the class 'y_num'.
    y = Y.get_y(y_num)
    # Check if 'theta' is a one-dimensional array.
    # If it is, convert it to a two-dimensional array by adding an extra dimension.
    if theta.ndim==1: theta = theta[:,np.newaxis]
    # Compute the cost function.
    cost_value = -( y*np.log(sigmoid(X@theta)) + (1-y)*np.log(1-sigmoid(X@theta)) ).mean()
    # Compute the regularization term.
    cost_regular = ((lamda/(2*m))*(theta[1:,:]**2).sum()
    # Compute the regularized cost function.
    cu_value = cost_value + cost_regular
    # Return the regularized cost function.
    return cu_value

# Define a function 'gradient' to compute the gradient of the regularized cost function for logistic regression.
def gradient(theta,X,Y,y_num,lamda=1):
    # 'm' is the number of training examples.
    m = X.shape[0]
    # 'y' is the target variable for the class 'y_num'.
    y = Y.get_y(y_num)
    # Check if 'theta' is a one-dimensional array.
    # If it is, convert it to a two-dimensional array by adding an extra dimension.
    if theta.ndim==1: theta = theta[:,np.newaxis]
    # Compute the gradient for the intercept term.
    theta_0 = (sigmoid(X@theta)-y).mean()
    # Compute the gradient for the other terms.
    theta = (1/m)*(X.T@(sigmoid(X@theta)-y)) + (lamda/m)*theta
    # Replace the gradient for the intercept term.
    theta[0,:] = theta_0
    # Return the gradient as a one-dimensional array.
    return theta.flatten()

In [95]:
# Initialize an empty dictionary 'theta_dic' to store the parameter vectors for each class.
theta_dic = {}

# Loop over the classes from 0 to 9.
for i in range(10):
    # Use the 'minimize' function from scipy's optimize module to find the parameters that minimize the cost function.
    # 'fun' is the cost function to be minimized.
    # 'args' are additional arguments to the cost function, which are 'X', 'Y', and 'i' in this case.
    # 'x0' is the initial guess for the parameters, which is 'theta' in this case.
    # 'method' is the optimization method to be used, which is 'Newton-CG' in this case.
    # 'jac' is the function that computes the gradient vector, which is 'gradient' in this case.
    res = optimize.minimize(cost, args=(X, Y, i), x0=theta, method='Newton-CG', jac=gradient)
    # Store the final parameter vector in 'theta_dic' with the class label as the key.
    theta_dic[i] = res.x

In [121]:
# Import the 'classification_report' function from sklearn's metrics module.
# This function builds a text report showing the main classification metrics.
from sklearn.metrics import classification_report

# Define a function 'y_pred' to predict the class labels for the examples in 'X'.
# 'theta_dic' is a dictionary containing the parameter vectors for each class.
# 'X' is the feature matrix.
def y_pred(theta_dic, X):
    # Convert 'theta_dic' to a data frame, then to a numpy array, and finally transpose it to get a matrix where each row is a parameter vector.
    theta_matrix = pd.DataFrame(theta_dic).to_numpy().T
    # Compute the product of 'theta_matrix' and 'X' transposed.
    theta_x = theta_matrix@X.T
    # Find the index of the maximum value in each column of 'theta_x'.
    # This is the predicted class label for each example in 'X'.
    y_index = np.argmax(theta_x,axis=0)
    # Return the predicted class labels.
    return y_index

# 'y_p' is assigned the predicted class labels for the examples in 'X'.
y_p = y_pred(theta_dic,X)

In [124]:
result = classification_report(raw_data["y"].flatten(),y_p)
print(result)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       500
           1       0.95      0.99      0.97       500
           2       0.95      0.92      0.93       500
           3       0.95      0.91      0.93       500
           4       0.95      0.95      0.95       500
           5       0.92      0.92      0.92       500
           6       0.97      0.98      0.97       500
           7       0.95      0.95      0.95       500
           8       0.93      0.92      0.92       500
           9       0.92      0.92      0.92       500

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000

